In [1]:
# Reference
# https://linuxhint.com/python_xml_to_dictionary/

In [4]:
base = './data/ori'

In [10]:
import os
import xmltodict
import math
import random
from copy import deepcopy

def convert2coco(base):
    
    # Open an xml file and convert it into the dictionary format
    def xml_to_dict(base, filename):
        with open(os.path.join(base, filename), "r") as xml_obj:
            my_dict = xmltodict.parse(xml_obj.read())
            xml_obj.close()
        return my_dict
    
    # Get filenames of xml files
    filenames = [filename for filename in os.listdir(base) if os.path.splitext(filename)[-1] == '.xml']
            
    # Get all classes in the dataset
    cls_dict = {}
    for filename in filenames:
        ann  = xml_to_dict(base, filename)['annotation']
        for obj in ann['object']:
            cls_name = obj['name']
            if cls_name not in cls_dict:
                cls_dict[cls_name] = len(cls_dict) + 1
    
    # Initialize the annotations dictionary
    annotations = {
        "type": "instances",
        "images": [],
        "categories": [],
        "annotations": []
    }
    img_id = 1; ann_id = 1
    
    # Convert to the COCO Object Detection format
    for filename in filenames:
        # Get annotations of an xml file
        ann  = xml_to_dict(base, filename)['annotation']
            
        for ann_obj in ann['object']:
            bbx = ann_obj['bndbox']
            xmin, ymin = int(bbx['xmin']), int(bbx['ymin'])
            xmax, ymax = int(bbx['xmax']), int(bbx['ymax'])
            dx = xmax - xmin
            dy = ymax - ymin
            
            annot = {
                "id": ann_id,
                "bbox": [xmin, ymin, dx, dy],
                "image_id": img_id,
                "category_id": cls_dict[ann_obj['name']],
                "segmentation": [],
                "area": dx*dy,
                "iscrowd": 0
            }
            annotations["annotations"].append(annot)
            ann_id = ann_id + 1
            
        size = ann['size']
        image = {
            "file_name": ann['filename'],
            "height":size['height'] ,
            "width": size['width'],
            "id": img_id
        }
        annotations["images"].append(image)
        img_id = img_id + 1
            
    for cls_name, cls_id in cls_dict.items():
        annotations["categories"].append({
            "supercategory": "none",
            "name": cls_name,
            "id": cls_id
        })
        
    return annotations

In [11]:
annotations = convert2coco(base)

In [44]:
def train_test_split(input_annotations, split_dictionary, max_iter=100):    
    total = sum([val for _, val in split_dictionary.items()])
    split_dict = {key: val/total for key, val in split_dictionary.items()}
    
    # Get the number of objects
    categories = [category['id'] for category in input_annotations['categories']]
    total_objects = {cat_id: 0 for cat_id in categories}
    
    def coco_to_img2annots(annotations):
        img2annots = {}

        num_obj_init = {category['id']: 0 for category in annotations['categories']}
        for image in annotations['images']:
            image_id = image['id']
            if image_id in img2annots:
                print('Error: There are duplicate image ids, please check your dataset.')
                return None
            img2annots[image_id] = {
                'description': deepcopy(image),
                'annotations': [],
                'num_objects': deepcopy(num_obj_init)
            }

        for annotation in annotations['annotations']:
            if annotation['image_id'] not in img2annots:
                print('Error: There exists an annotation where image_id does not exist in annotations["images"]')
                return None
            img2annots[annotation['image_id']]['annotations'].append(annotation)

        return img2annots
    
    img2annots = coco_to_img2annots(input_annotations)
    print(img2annots[10])
    return
    
    
    for key, val in img2annots.items():
        for cat_id, cat_n in val['num_objects'].items():
            total_objects[cat_id] = total_objects[cat_id] + cat_n
    
    # Get the spit_size for every set
    total_img = len(img2annots.keys())
    split_size_img = {}
    for key1, val1 in split_dict.items():
        split_size_img[key1] = math.ceil(val1*total_img)
            
    # Get the index_mapping for each set
    split_img_dict = {}
    start_idx = 0
    for key, val in split_size_img.items():
        split_img_dict[key] = [start_idx, min(start_idx + val, total_img)]
        start_idx = start_idx + val
        
    # Calculate the percentage of objects w.r.t to total objects
    def calculate_object(data_dict, total_objects):
        count = {key: 0 for key, _ in total_objects.items()}
        for key, val in data_dict.items():
            for ann in val['annotations']:
                category_id = ann['category_id']
                count[category_id] = count[category_id] + 1
        for key, val in total_objects.items():
            count[key] = count[key] / val
        return count
            
    # Optimization
    img_name = list(img2annots.keys())
    obj_counts = {}
    best_error = 1.
    for i in range(max_iter):
        random.shuffle(img_name)
        
        for key, val in split_img_dict.items():
            obj_dict = {name: img2annots[name] for name in img_name[val[0]:val[1]]}
            obj_counts[key] = calculate_object(obj_dict, total_objects)
            
        error = 0
        for key1, val1 in split_dictionary.items():
            for key2, val2 in obj_counts[key1].items():
                error = error + (val1-val2)**2
        
        if error < best_error:
            best_error = deepcopy(error)
            best_img_name_seq = deepcopy(img_name)
    
    # Split the dataset
    annotations = {
        
    }
    for key1, val1 in split_img_dict.items():
        obj_dict = {name: img2annots[name] for name in best_img_name_seq[val1[0]:val1[1]]}
        annotations[key1] = {
            'type': input_annotations['type'],
            'categories': input_annotations['categories'],
            'images': [],
            'annotations': []
        }
        for key2, val2 in obj_dict.items():
            annotations[key1]['images'].append(val2['data'])
            annotations[key1]['annotations'].extend(val2['annotations'])
    return annotations

In [45]:
split_dictionary = {
    'train': 0.60,
    'val': 0.20,
    'test': 0.20
}

ann_split = train_test_split(annotations, split_dictionary)

{'description': {'file_name': 'IMG01399.JPG', 'height': '1080', 'width': '1920', 'id': 10}, 'annotations': [{'id': 1271, 'bbox': [153, 63, 39, 39], 'image_id': 10, 'category_id': 1, 'segmentation': [], 'area': 1521, 'iscrowd': 0}, {'id': 1272, 'bbox': [173, 89, 39, 39], 'image_id': 10, 'category_id': 1, 'segmentation': [], 'area': 1521, 'iscrowd': 0}, {'id': 1273, 'bbox': [141, 97, 39, 39], 'image_id': 10, 'category_id': 1, 'segmentation': [], 'area': 1521, 'iscrowd': 0}, {'id': 1274, 'bbox': [155, 126, 41, 52], 'image_id': 10, 'category_id': 1, 'segmentation': [], 'area': 2132, 'iscrowd': 0}, {'id': 1275, 'bbox': [230, 83, 44, 44], 'image_id': 10, 'category_id': 1, 'segmentation': [], 'area': 1936, 'iscrowd': 0}, {'id': 1276, 'bbox': [367, 49, 50, 51], 'image_id': 10, 'category_id': 1, 'segmentation': [], 'area': 2550, 'iscrowd': 0}, {'id': 1277, 'bbox': [392, 18, 39, 35], 'image_id': 10, 'category_id': 1, 'segmentation': [], 'area': 1365, 'iscrowd': 0}, {'id': 1278, 'bbox': [424, 1, 

In [8]:
b

2